# Lohnsteuer BMF API

This notebook makes use of the API by the [German Ministry of Finance (BMF)](https://www.bmf-steuerrechner.de/interface/einganginterface.xhtml). When calling the function `call_lohnsteuer_api` 
- a set of test households is generated
- the variable `column` (either `lohnst_m` or `soli_st_lohnst_m`) is calculated using the BMF API
- the output is compared against `gettsim` output

This notebook is meant to help `gettsim` developers in the realm of `lohnsteuer`. As the API address is not stable over time and documentation does not always reflect its current state, we decided not to include it as a regular unit test for `gettsim`.

## Basic imports

In [ ]:
import urllib.request as mybrowser
from xml.etree import ElementTree

import numpy as np
import pandas as pd
from _gettsim.interface import compute_taxes_and_transfers
from _gettsim.policy_environment import set_up_policy_environment
from pandas.testing import assert_series_equal

## Definition of internal functions

In [ ]:
def get_bmf_url(base, specs):
    """Formatting of URL.

    Parameters
    ----------
    base : str
        the base URL from BMF for the given year
    specs : dict
        Dictionary of arguments for the URL call

    Returns
    -------
    url: str
        the URL we want to send

    """
    url = base
    for spec in specs:
        url = url + f"&{spec}={specs[spec]}"
    return url

In [ ]:
def get_xml(url):
    """Send URL to BMF and obtain xml.

    Parameters
    ----------
    url : str
        the url specified according to BMF API

    Returns
    -------
    xml_ugly: bytes
        the server response

    """
    # say hello
    myheader = {
        "User-Agent": """gettsim development: https://github.com/iza-institute-of-labor-economics/gettsim"""  # noqa: E501
    }
    request = mybrowser.Request(url, headers=myheader)
    response = mybrowser.urlopen(request)
    xml_ugly = response.read()

    return xml_ugly

In [ ]:
def get_bmf_data(url_base, specs, out_definitions):
    """Format URL, send it to BMF and format output.

    Parameters
    ----------
    url_base : str
        base URL
    specs : dict
    out_definitions : dict

    Returns
    -------
    df: pandas.DataFrame

    url: str

    """

    out_df = pd.DataFrame(out_definitions, index=["definition"]).T

    url = get_bmf_url(url_base, specs)

    df = pd.DataFrame(columns=["name", "value", "type"])
    # get xml results
    xml_ugly = get_xml(url)
    if "Der Zugriffscode ist abgelaufen" in str(xml_ugly):
        return "Error"

    xml_raw = ElementTree.fromstring(xml_ugly)
    # put information into DataFrame
    for index, child in enumerate(xml_raw.iter("ausgabe")):
        df = pd.concat([df, pd.DataFrame(child.attrib, index=[index])])

    return df.set_index("name").join(out_df)

In [ ]:
def bmf_collect(  # noqa: PLR0913
    inc, outvar, n_kinder, stkl, jahr, zusatzbeitrag, faktorverfahren=0, faktor="1,0000"
):
    """Creates an URL for the API of the official calculator by the German Ministry of
    Finance (BMF), documented at: https://www.bmf-
    steuerrechner.de/interface/einganginterface.xhtml this url is called and the results
    are returned.

    Returns
    -------

    income tax due, depending on the value of outvar

    """
    url_base = f"http://www.bmf-steuerrechner.de/interface/{jahr}Version1.xhtml?"
    # ATTENTION: This bit changes on a yearly basis
    url_base += "code=ext2023"

    # Possible inputs:
    # https://www.bundesfinanzministerium.de/Content/DE/Downloads/Steuern/Steuerarten/Lohnsteuer/Programmablaufplan/2020-11-09-PAP-2021-anlage-1.pdf?__blob=publicationFile&v=https://www.bundesfinanzministerium.de/Content/DE/Downloads/Steuern/Steuerarten/Lohnsteuer/Programmablaufplan/2020-11-09-PAP-2021-anlage-1.pdf?__blob=publicationFile&v=2
    # Faktor: eingetragener Faktor
    # LZZ = 1: yearly income, 2: monthly
    # PVZ (1/0): PV-Zusatzbeitrag für Kinderlose
    # RE4: Steuerpflichtiger Arbeitslohn (in Cent!!!)
    # KVZ: GKV Zusatzbeitrag in %
    #
    # Speficy the call. everything that is not specified is treated as a zero value.

    if stkl == 4:
        kinderfb = n_kinder / 2
    else:
        kinderfb = n_kinder

    kinderlos = int(n_kinder == 0)

    specs = {
        "RE4": inc * 100,
        "AF": faktorverfahren,
        "F": faktor,
        "LZZ": 2,  # i.e. income is monthly
        "STKL": stkl,
        "ZKF": kinderfb,
        "KVZ": f"{zusatzbeitrag*100:.2f}".replace(".", ","),
        "PVZ": kinderlos,
    }
    out_definitions = {
        "BK": "Bemessungsgrundlage für die Kirchenlohnsteuer in Cent",
        "BKS": """Bemessungsgrundlage der sonstigen Bezüge
                (ohne Vergütung für mehrjährige Tätigkeit) für die Kirchenlohnsteuer
                in Cent""",
        "BKV": """Bemessungsgrundlage der Vergütung für mehrjährige Tätigkeit
                  für die Kirchenlohnsteuer in Cent""",
        "LSTLZZ": "Für den Lohnzahlungszeitraum einzubehaltende Lohnsteuer in Cent",
        "SOLZLZZ": """Für den Lohnzahlungszeitraum einzubehaltender
        Solidaritätszuschlag in Cent""",
        "VKVLZZ": """Für den Lohnzahlungszeitraum berücksichtigte Beiträge des
        Arbeitnehmers zur privaten Basis Krankenversicherung und privaten Pflege
        Pflichtversicherung (ggf. auch die Mindestvorsorgepauschale) in Cent beim
        laufenden Arbeitslohn. Für Zwecke der Lohnsteuerbescheinigung sind die
        einzelnen Ausgabewerte außerhalb des eigentlichen
        Lohnsteuerberechnungsprogramms zu addieren;
         hinzuzurechnen sind auch die Ausgabewerte VKVSONST.""",
        "VFRB": """Verbrauchter Freibetrag bei Berechnung des
        laufenden Arbeitslohns, in Cent""",
    }

    bmf_out = get_bmf_data(url_base, specs, out_definitions)
    out = bmf_out["value"].astype(int)
    # Divide by 100 to get Euro
    return out[outvar] / 100

In [ ]:
def gen_lohnsteuer_test(year: int, soz_vers_params: dict):
    """Calls the BMF API to generate correct lohnsteuer payments."""

    hh = pd.DataFrame(
        {
            "p_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
            "tu_id": [1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 6],
            "bruttolohn_m": [2000, 3000, 4000, 5000, 0, 2000, 0, 0, 3000, 0, 7500],
            "alter": [30, 30, 40, 40, 50, 30, 5, 2, 40, 12, 50],
            "kind": [
                False,
                False,
                False,
                False,
                False,
                False,
                True,
                True,
                False,
                True,
                False,
            ],
            "steuerklasse": [1, 4, 4, 3, 5, 2, 1, 1, 2, 2, 1],
            "year": [year] * 11,
        }
    )
    hh["child_num_kg"] = hh.groupby("tu_id")["kind"].transform("sum")

    # Get correct lohnsteuer from German Ministry of Finance
    hh["lohnst_m"] = np.vectorize(bmf_collect)(
        inc=hh["bruttolohn_m"],
        outvar="LSTLZZ",
        n_kinder=hh["child_num_kg"],
        stkl=hh["steuerklasse"],
        jahr=hh["year"],
        zusatzbeitrag=soz_vers_params["beitr_satz"]["ges_krankenv"][
            "mean_zusatzbeitrag"
        ],
    )

    hh["soli_st_lohnst_m"] = np.vectorize(bmf_collect)(
        hh["bruttolohn_m"],
        outvar="SOLZLZZ",
        n_kinder=hh["child_num_kg"],
        stkl=hh["steuerklasse"],
        jahr=hh["year"],
        zusatzbeitrag=soz_vers_params["beitr_satz"]["ges_krankenv"][
            "mean_zusatzbeitrag"
        ],
    )

    return hh

In [ ]:
def call_lohnsteuer_api(year: int, column: str):
    """This test compares gettsim values for lohnsteuer against the API by the German
    Ministry of Finance.

    The API address is not stable over time. For this reason, the test is skipped by
    default


    """
    policy_params, policy_functions = set_up_policy_environment(date=year)

    year_data = gen_lohnsteuer_test(
        year, soz_vers_params=policy_params["soz_vers_beitr"]
    ).reset_index(drop=True)
    df = year_data.copy()
    df["alleinerz"] = df["steuerklasse"] == 2
    df["wohnort_ost"] = False
    df["jahr_renteneintr"] = 2060
    df["hat_kinder"] = df.groupby("tu_id")["kind"].transform("sum") > 0
    df["in_ausbildung"] = df["kind"]
    df["arbeitsstunden_w"] = 40.0 * ~df["kind"]

    result = compute_taxes_and_transfers(
        data=df.drop(columns=["lohnst_m", "soli_st_lohnst_m"]),
        params=policy_params,
        functions=policy_functions,
        targets=[column],
    )

    assert_series_equal(
        result[column], year_data[column], check_exact=False, atol=2, check_dtype=False
    )

## Call the API

In [ ]:
call_lohnsteuer_api(year=2022, column="lohnst_m")